In [107]:
import os
import os.path
import glob
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import random
import matplotlib.pyplot as plt
import json
from tensorflow.keras.layers import Flatten, Input, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalMaxPool2D, \
    GlobalAveragePooling2D, Dropout, Add
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from keras import backend as K
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [84]:
directory = './regression_modeldata1'

with open('./groundtruth.json') as json_file:
    gtdata = json.load(json_file)
subdirs = os.listdir(directory)

finalX_img = []
finalX_num = []
finalyLeft = []
finalyRight = []

for i in range(len(subdirs)):
    dir2pull = directory + '/' + subdirs[i] + '/'
    print(dir2pull)

    lefty = gtdata[subdirs[i]]['leftlength']
    righty = gtdata[subdirs[i]]['rightlength']

    for file in glob.glob(dir2pull + '*.csv'):
        data2read = file.replace("\\", '/')
        # print(data2read)
        # print(data2read)
        singledata = pd.read_csv(data2read, header=None)
        singledata = np.array(singledata)

        dataarray = []

        for j in range(singledata.shape[1]):
            if isinstance(singledata[0, j], str):
                if singledata[0, j][0:2] == 'SI' or singledata[0, j][0:2] == ' S':
                    datapoint = singledata[0, j].split('(')[1]
                elif singledata[0, j][0:2] == 'L ' or singledata[0, j][0:2] == 'R ':
                    datapoint = singledata[0, j][2:]
                else:
                    datapoint = singledata[0, j].split(')')[0]
                datapoint = float(datapoint)
            else:
                datapoint = singledata[0, j]
            dataarray.append(datapoint)
        procdata = np.array(dataarray)
        # print(procdata.shape)

        if procdata.shape[0] != 60:
            continue

        finalX_num.append(procdata)
        finalyLeft.append(lefty + random.uniform(-0.5, 0.5))
        finalyRight.append(righty + random.uniform(-0.5, 0.5))

        imgpath = data2read.split('.cs')[0]

        if os.path.isfile(imgpath):
            image = cv2.imread(imgpath)
        else:
            image = cv2.imread(imgpath + '.jpg')

        # print(image.shape)

        finalX_img.append(image)

# print(len(finalX))

finalX_img = np.array(finalX_img)
print(finalX_img.shape)

finalX_num = np.array(finalX_num)
print(finalX_num.shape)

finalyLeft = np.array(finalyLeft)
finalyRight = np.array(finalyRight)


def shuffle_in_unison_four(a, b, c, d):
    n_elem = a.shape[0]
    indeces = np.random.choice(n_elem, size=n_elem, replace=False)
    return a[indeces], b[indeces], c[indeces], d[indeces]


# def createinputForModel():


X_img, X_num, yL, yR = shuffle_in_unison_four(finalX_img, finalX_num, finalyLeft, finalyRight)

./regression_modeldata1/alex/
./regression_modeldata1/senthil/
./regression_modeldata1/sharma/
./regression_modeldata1/user1/
./regression_modeldata1/vaneesh/
(126, 1792, 828, 3)
(126, 60)


In [85]:
X_img_train = X_img[:100]
X_img_test = X_img[100:]
X_num_train = X_num[:100]
X_num_test = X_num[100:]
yL_train = yL[:100]
yL_test = yL[100:]
yR_train = yR[:100]
yR_test = yR[100:]

X_img_train = X_img
X_num_train = X_num
yL_train = yL
yR_train = yR

In [86]:
X_num_train.shape

(126, 60)

In [87]:
X_num_train = X_num_train.reshape(X_num_train.shape[0],X_num_train.shape[1],1)
#X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_num_test = X_num_test.reshape(X_num_test.shape[0],X_num_test.shape[1],1)

In [90]:
print(X_num_train.shape, X_num_test.shape)


(126, 60, 1) (26, 60, 1)


In [91]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [92]:
# Setting tensorflow graphs

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [93]:
input1 = Input(shape=(None, None, 3))
input2 = Input(shape=(None,1))

In [94]:
input1.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(3)])

In [95]:
input2.shape

TensorShape([Dimension(None), Dimension(None), Dimension(1)])

In [96]:
base_model = VGG16(include_top=False, input_tensor=input1)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)

merge = Add()([x, input2])


left = Dense(units=1024, activation='relu')(merge)
left = Dense(units=512, activation='relu')(left)
left = Dense(units=128, activation='relu')(left)

left_length = Dense(units=1, activation='linear', name='left_length')(left)

right = Dense(units=1024, activation='relu')(merge)
right = Dense(units=512, activation='relu')(right)
right = Dense(units=128, activation='relu')(right)

right_length = Dense(units=1, activation='linear', name='right_length')(right)

model = Model(inputs=[input1, input2], outputs=[left_length, right_length])
for layer in base_model.layers:
    layer.trainable = False

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 6 1792        input_14[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, None, None, 6 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, None, None, 6 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [98]:

print(X_img_train.shape)

# image_train = X_img_train / 255

yL_train = np.expand_dims(yL_train,axis=1)
yR_train = np.expand_dims(yR_train,axis=1)

yL_train = np.expand_dims(yL_train,axis=2)
yR_train = np.expand_dims(yR_train,axis=2)


print(yL_train.shape)

print(X_img_train.shape)
print(X_num_train.shape)
print(yL_train.shape)
print(yR_train.shape)


(126, 1792, 828, 3)
(126, 1, 1)
(126, 1792, 828, 3)
(126, 60, 1)
(126, 1, 1)
(126, 1, 1)


In [106]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model.compile(loss={'left_length': 'mse', 'right_length': 'mse'}, optimizer='adam',
              metrics={'left_length': root_mean_squared_error, 'right_length': root_mean_squared_error})

# history = model.fit(X_train, y_train, batch_size = 50, validation_split = 0.2, epochs = 100, verbose = 0)

model.fit([X_img_train, X_num_train], [yL_train, yR_train], batch_size=2, validation_split=0.1, epochs=200, verbose=1,callbacks=[tensorboard_cb])
#model.fit([X_CNN1, X_CNN2], y)

print('-----done--------')

Train on 113 samples, validate on 13 samples
Epoch 1/200
113/113 [==============================] - 33s 296ms/step - loss: 164673.3174 - left_length_loss: 74218.1859 - right_length_loss: 90455.1289 - left_length_root_mean_squared_error: 216.5948 - right_length_root_mean_squared_error: 212.8094 - val_loss: 11670.2332 - val_left_length_loss: 6848.3686 - val_right_length_loss: 4821.8647 - val_left_length_root_mean_squared_error: 80.9323 - val_right_length_root_mean_squared_error: 68.0267
Epoch 2/200
113/113 [==============================] - 33s 290ms/step - loss: 5705.6596 - left_length_loss: 3015.9037 - right_length_loss: 2689.7559 - left_length_root_mean_squared_error: 53.2970 - right_length_root_mean_squared_error: 50.6670 - val_loss: 5002.8865 - val_left_length_loss: 2476.3661 - val_right_length_loss: 2526.5203 - val_left_length_root_mean_squared_error: 46.7364 - val_right_length_root_mean_squared_error: 47.2823
Epoch 3/200
113/113 [==============================] - 33s 293ms/step - 

Epoch 20/200
113/113 [==============================] - 36s 317ms/step - loss: 2378.4361 - left_length_loss: 1330.9984 - right_length_loss: 1047.4377 - left_length_root_mean_squared_error: 35.5963 - right_length_root_mean_squared_error: 31.8790 - val_loss: 2344.0214 - val_left_length_loss: 1215.6885 - val_right_length_loss: 1128.3329 - val_left_length_root_mean_squared_error: 33.9817 - val_right_length_root_mean_squared_error: 32.8049
Epoch 21/200
113/113 [==============================] - 37s 325ms/step - loss: 2047.1360 - left_length_loss: 1024.2461 - right_length_loss: 1022.8899 - left_length_root_mean_squared_error: 31.5734 - right_length_root_mean_squared_error: 31.5062 - val_loss: 2403.0327 - val_left_length_loss: 1206.6661 - val_right_length_loss: 1196.3667 - val_left_length_root_mean_squared_error: 34.1964 - val_right_length_root_mean_squared_error: 34.1101
Epoch 22/200
113/113 [==============================] - 35s 310ms/step - loss: 2130.0818 - left_length_loss: 1124.7767 - r

Epoch 39/200
113/113 [==============================] - 43s 380ms/step - loss: 2230.4513 - left_length_loss: 1031.5199 - right_length_loss: 1198.9314 - left_length_root_mean_squared_error: 31.6801 - right_length_root_mean_squared_error: 33.2844 - val_loss: 6065.4580 - val_left_length_loss: 1123.9820 - val_right_length_loss: 4941.4760 - val_left_length_root_mean_squared_error: 32.9995 - val_right_length_root_mean_squared_error: 70.2654
Epoch 40/200
113/113 [==============================] - 43s 377ms/step - loss: 2509.7759 - left_length_loss: 1026.3991 - right_length_loss: 1483.3768 - left_length_root_mean_squared_error: 31.6932 - right_length_root_mean_squared_error: 37.2249 - val_loss: 2333.3046 - val_left_length_loss: 1226.1448 - val_right_length_loss: 1107.1598 - val_left_length_root_mean_squared_error: 34.3473 - val_right_length_root_mean_squared_error: 32.6176
Epoch 41/200
113/113 [==============================] - 43s 380ms/step - loss: 2098.0153 - left_length_loss: 1042.9855 - r

Epoch 58/200
113/113 [==============================] - 42s 371ms/step - loss: 2003.5390 - left_length_loss: 1047.3759 - right_length_loss: 956.1632 - left_length_root_mean_squared_error: 31.8613 - right_length_root_mean_squared_error: 30.4789 - val_loss: 2130.8666 - val_left_length_loss: 1113.6539 - val_right_length_loss: 1017.2126 - val_left_length_root_mean_squared_error: 32.7307 - val_right_length_root_mean_squared_error: 31.0478
Epoch 59/200
113/113 [==============================] - 44s 392ms/step - loss: 1974.0646 - left_length_loss: 1033.0026 - right_length_loss: 941.0620 - left_length_root_mean_squared_error: 31.6489 - right_length_root_mean_squared_error: 30.1638 - val_loss: 2440.4982 - val_left_length_loss: 1282.3039 - val_right_length_loss: 1158.1943 - val_left_length_root_mean_squared_error: 35.2931 - val_right_length_root_mean_squared_error: 33.0935
Epoch 60/200
113/113 [==============================] - 44s 389ms/step - loss: 1975.4731 - left_length_loss: 1014.5601 - rig

Epoch 77/200
113/113 [==============================] - 43s 382ms/step - loss: 1909.6491 - left_length_loss: 935.3365 - right_length_loss: 974.3126 - left_length_root_mean_squared_error: 30.0724 - right_length_root_mean_squared_error: 30.7044 - val_loss: 2097.2522 - val_left_length_loss: 1064.8146 - val_right_length_loss: 1032.4375 - val_left_length_root_mean_squared_error: 31.9774 - val_right_length_root_mean_squared_error: 31.4837
Epoch 78/200
113/113 [==============================] - 43s 384ms/step - loss: 1889.1525 - left_length_loss: 942.9991 - right_length_loss: 946.1534 - left_length_root_mean_squared_error: 30.1903 - right_length_root_mean_squared_error: 30.2497 - val_loss: 2057.1051 - val_left_length_loss: 1029.7718 - val_right_length_loss: 1027.3333 - val_left_length_root_mean_squared_error: 31.3282 - val_right_length_root_mean_squared_error: 31.3594
Epoch 79/200
113/113 [==============================] - 43s 382ms/step - loss: 1838.5912 - left_length_loss: 911.3616 - right_

...

Epoch 115/200
113/113 [==============================] - 48s 421ms/step - loss: 1901.3528 - left_length_loss: 975.0619 - right_length_loss: 926.2910 - left_length_root_mean_squared_error: 30.7206 - right_length_root_mean_squared_error: 29.9712 - val_loss: 2061.4251 - val_left_length_loss: 1053.4069 - val_right_length_loss: 1008.0181 - val_left_length_root_mean_squared_error: 31.6887 - val_right_length_root_mean_squared_error: 31.0984
Epoch 116/200
113/113 [==============================] - 47s 416ms/step - loss: 1869.0146 - left_length_loss: 935.8208 - right_length_loss: 933.1938 - left_length_root_mean_squared_error: 30.0839 - right_length_root_mean_squared_error: 30.0469 - val_loss: 2169.1227 - val_left_length_loss: 1169.6478 - val_right_length_loss: 999.4748 - val_left_length_root_mean_squared_error: 33.5314 - val_right_length_root_mean_squared_error: 30.9788
Epoch 117/200
113/113 [==============================] - 52s 458ms/step - loss: 1865.0870 - left_length_loss: 942.8701 - righ

Epoch 134/200
113/113 [==============================] - 44s 387ms/step - loss: 1842.8077 - left_length_loss: 919.0800 - right_length_loss: 923.7277 - left_length_root_mean_squared_error: 29.7994 - right_length_root_mean_squared_error: 29.9185 - val_loss: 2029.0319 - val_left_length_loss: 1046.4584 - val_right_length_loss: 982.5735 - val_left_length_root_mean_squared_error: 31.4650 - val_right_length_root_mean_squared_error: 30.6104
Epoch 135/200
113/113 [==============================] - 44s 390ms/step - loss: 1903.4761 - left_length_loss: 942.5735 - right_length_loss: 960.9027 - left_length_root_mean_squared_error: 30.1261 - right_length_root_mean_squared_error: 30.4459 - val_loss: 2122.7361 - val_left_length_loss: 1061.1623 - val_right_length_loss: 1061.5738 - val_left_length_root_mean_squared_error: 31.7175 - val_right_length_root_mean_squared_error: 31.6975
Epoch 136/200
113/113 [==============================] - 44s 393ms/step - loss: 1871.2222 - left_length_loss: 944.7385 - righ

Epoch 153/200
113/113 [==============================] - 43s 381ms/step - loss: 1831.0671 - left_length_loss: 921.0304 - right_length_loss: 910.0367 - left_length_root_mean_squared_error: 29.7579 - right_length_root_mean_squared_error: 29.6477 - val_loss: 2182.0341 - val_left_length_loss: 1082.3576 - val_right_length_loss: 1099.6765 - val_left_length_root_mean_squared_error: 32.5888 - val_right_length_root_mean_squared_error: 32.7658
Epoch 154/200
113/113 [==============================] - 44s 391ms/step - loss: 1832.0991 - left_length_loss: 923.7590 - right_length_loss: 908.3401 - left_length_root_mean_squared_error: 29.8055 - right_length_root_mean_squared_error: 29.5286 - val_loss: 2016.1774 - val_left_length_loss: 1017.6177 - val_right_length_loss: 998.5596 - val_left_length_root_mean_squared_error: 31.1431 - val_right_length_root_mean_squared_error: 30.8855
Epoch 155/200
113/113 [==============================] - 45s 394ms/step - loss: 1781.7177 - left_length_loss: 894.3206 - righ

Epoch 172/200
113/113 [==============================] - 44s 387ms/step - loss: 1809.1651 - left_length_loss: 882.0099 - right_length_loss: 927.1551 - left_length_root_mean_squared_error: 29.1838 - right_length_root_mean_squared_error: 30.0030 - val_loss: 2038.6957 - val_left_length_loss: 984.4687 - val_right_length_loss: 1054.2270 - val_left_length_root_mean_squared_error: 30.6918 - val_right_length_root_mean_squared_error: 31.5828
Epoch 173/200
113/113 [==============================] - 44s 387ms/step - loss: 1825.2023 - left_length_loss: 933.1169 - right_length_loss: 892.0853 - left_length_root_mean_squared_error: 29.8001 - right_length_root_mean_squared_error: 29.2668 - val_loss: 2041.0857 - val_left_length_loss: 1056.3158 - val_right_length_loss: 984.7699 - val_left_length_root_mean_squared_error: 31.7543 - val_right_length_root_mean_squared_error: 30.6574
Epoch 174/200
113/113 [==============================] - 44s 392ms/step - loss: 1844.1509 - left_length_loss: 930.3749 - right

Epoch 191/200
113/113 [==============================] - 43s 383ms/step - loss: 1876.1087 - left_length_loss: 986.1010 - right_length_loss: 890.0077 - left_length_root_mean_squared_error: 31.0053 - right_length_root_mean_squared_error: 29.4741 - val_loss: 2003.3512 - val_left_length_loss: 1032.8494 - val_right_length_loss: 970.5018 - val_left_length_root_mean_squared_error: 31.3861 - val_right_length_root_mean_squared_error: 30.6292
Epoch 192/200
113/113 [==============================] - 43s 382ms/step - loss: 1733.6503 - left_length_loss: 886.6725 - right_length_loss: 846.9779 - left_length_root_mean_squared_error: 29.4203 - right_length_root_mean_squared_error: 28.8106 - val_loss: 2013.6870 - val_left_length_loss: 1049.1028 - val_right_length_loss: 964.5841 - val_left_length_root_mean_squared_error: 31.6028 - val_right_length_root_mean_squared_error: 30.4010
Epoch 193/200
113/113 [==============================] - 44s 387ms/step - loss: 1750.2945 - left_length_loss: 884.3751 - right

In [83]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 6440.

In [117]:
# prototyping image generator that takes in 2 inputs
# https://github.com/keras-team/keras/issues/3386

def customDataAugGenerator(X_img,X_num,y_left, y_right, batch_size):
    
    while True:
        
        idx = np.random.permutation(X_img.shape[0])
        
        datagen = ImageDataGenerator(
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=5, #180,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.1, #0.1,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.1, #0.1,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=False,  # randomly flip images
                vertical_flip=False
        )
        
        batches = datagen.flow(X_img[idx], y_left[idx], batch_size=batch_size, shuffle=False)
        idx0 = 0
        for batch in batches:
            idx1 = idx0 + batch[0].shape[0]
            
            yield [batch[0], X_num[idx[idx0:idx1]]],[batch[1],y_right[idx[idx0:idx1]]]
            
            idx0 = idx1
            if idx1 >= X_img.shape[0]:
                break


In [121]:
datagenerator = customDataAugGenerator(X_img_train,X_num_train,yL_train,yR_train,2)

model.compile(loss={'left_length': 'mse', 'right_length': 'mse'}, optimizer='adam',
              metrics={'left_length': root_mean_squared_error, 'right_length': root_mean_squared_error})

#model.fit(datagenerator, batch_size=2, validation_split=0.1, epochs=200, verbose=1,callbacks=[tensorboard_cb])

model.fit_generator(datagenerator, validation_steps=True,steps_per_epoch=len(X_img_train)/2,epochs=200, verbose=1,callbacks=[tensorboard_cb])


Epoch 1/200
63/63 [==============================] - 41s 658ms/step - loss: 10275.1753 - left_length_loss: 2089.6643 - right_length_loss: 8185.5109 - left_length_root_mean_squared_error: 41.3063 - right_length_root_mean_squared_error: 64.3375
Epoch 2/200
63/63 [==============================] - 43s 684ms/step - loss: 2422.1371 - left_length_loss: 1259.4280 - right_length_loss: 1162.7091 - left_length_root_mean_squared_error: 34.3365 - right_length_root_mean_squared_error: 33.4310
Epoch 3/200
63/63 [==============================] - 52s 829ms/step - loss: 2625.3820 - left_length_loss: 967.7772 - right_length_loss: 1657.6048 - left_length_root_mean_squared_error: 30.7400 - right_length_root_mean_squared_error: 38.9652
Epoch 4/200
63/63 [==============================] - 54s 863ms/step - loss: 1954.7645 - left_length_loss: 998.0224 - right_length_loss: 956.7421 - left_length_root_mean_squared_error: 31.1491 - right_length_root_mean_squared_error: 30.5295
Epoch 5/200
63/63 [===============

MemoryError: Unable to allocate array with shape (126, 1792, 828, 3) and data type float32